# Example 1: Image Processing
## Detects the edges of an HDMI input and displays the output through HDMI

In [1]:
!pip install pynq opencv-python

^C


In [ ]:
from pynq import Overlay
import cv2
from pynq.lib.video import HDMIIn, HDMIOut

# Load the base overlay
base = Overlay('base.bit')

# Initialize HDMI input and output
hdmi_in = HDMIIn(base.video.hdmi_in)
hdmi_out = HDMIOut(base.video.hdmi_out)
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode)
hdmi_in.start()
hdmi_out.start()

# Capture a frame
frame = hdmi_in.readframe()

# Convert the frame to grayscale
gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Apply edge detection
edges = cv2.Canny(gray_frame, 100, 200)

# Display the processed frame
hdmi_out.writeframe(edges)

# Example 2: Velocity Verlet Algorithm

## This is a commonly known algorithim that solves for Newton's laws of motions. This is edited to use the PYQN board from my code for PHSX 521 solving for energy of a spring.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pynq import Overlay, allocate

# Load the base overlay
overlay = Overlay('base.bit')

# Constants
k = 1  # Spring constant
m = 1  # Mass

# Time parameters
dt = 0.01  # Time step
t_max = 10  # Maximum time

# Damping Gamma coefficients for different regimes
damping_coeffs = {
    'No Damping': 0,
    'Underdamping': 0.1,
    'Critical Damping': 2 * np.sqrt(k / m),
    'Overdamping': 1
}

# Allocate buffers for PYNQ
position_buf = allocate(shape=(1,), dtype=np.float32)
velocity_buf = allocate(shape=(1,), dtype=np.float32)
acceleration_buf = allocate(shape=(1,), dtype=np.float32)
new_position_buf = allocate(shape=(1,), dtype=np.float32)
new_velocity_buf = allocate(shape=(1,), dtype=np.float32)
new_acceleration_buf = allocate(shape=(1,), dtype=np.float32)

# Function to calculate position and velocity using Velocity Verlet algorithm
def velocity_verlet(position, velocity, gamma, dt):
    acceleration = -k * position / m - gamma * velocity
    position_new = position + velocity * dt + 0.5 * acceleration * dt**2
    acceleration_new = -k * position_new / m - gamma * velocity
    velocity_new = velocity + 0.5 * (acceleration + acceleration_new) * dt
    return position_new, velocity_new

# Simulate and plot for each damping regime
plt.figure(figsize=(12, 10))
for i, (regime, gamma) in enumerate(damping_coeffs.items()):
    # Initialize arrays
    time = np.arange(0, t_max, dt)
    position = np.zeros_like(time)
    velocity = np.zeros_like(time)
    energy = np.zeros_like(time)

    # Initial conditions
    position[0] = 1
    velocity[0] = 0

    # Simulate using Velocity Verlet algorithm
    for j in range(1, len(time)):
        position[j], velocity[j] = velocity_verlet(position[j-1], velocity[j-1], gamma, dt)
        energy[j] = 0.5 * m * velocity[j]**2 + 0.5 * k * position[j]**2 
    
    # Plot position vs time
    plt.subplot(2, 2, i + 1)
    plt.plot(time, position)
    plt.xlabel('Time')
    plt.ylabel('Position')
    plt.title(f'{regime} - Position vs Time')

plt.tight_layout()
plt.show()

# Example 3: YOLOv5 Model using SSD with Tensorflow

In [ ]:
import torch

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x

# Perform object detection
img = 'https://ultralytics.com/images/zidane.jpg'
results = model(img)

# Print results
results.print()

# Show results
results.show()

In [ ]:
import cv2
import torch
from pynq import Overlay

# Load the base overlay
overlay = Overlay('base.bit')

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x

# Initialize webcam
cap = cv2.VideoCapture(0)  # 0 is the ID of the default camera

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image")
        break

    # Perform object detection
    results = model(frame)

    # Draw bounding boxes and labels on the frame
    for result in results.xyxy[0].cpu().numpy():
        x1, y1, x2, y2, conf, cls = result
        label = f'{model.names[int(cls)]} {conf:.2f}'
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Object Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

# Example 4: Financial Visualizer

In [ ]:
!pip install pynq pandas numpy matplotlib yfinance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from pynq import Overlay

# Load the base overlay
overlay = Overlay('base.bit')

# Download historical stock data
def download_stock_data(ticker, period='1y'):
    stock_data = yf.download(ticker, period=period)
    return stock_data

# Calculate moving averages
def calculate_moving_averages(stock_data, short_window=50, long_window=200):
    stock_data['50_MA'] = stock_data['Close'].rolling(window=short_window).mean()
    stock_data['200_MA'] = stock_data['Close'].rolling(window=long_window).mean()
    return stock_data

# Visualize the stock price and moving averages
def plot_stock_data(stock_data, ticker):
    plt.figure(figsize=(14, 7))
    plt.plot(stock_data['Close'], label='Close Price')
    plt.plot(stock_data['50_MA'], label='50-Day Moving Average')
    plt.plot(stock_data['200_MA'], label='200-Day Moving Average')
    plt.title(f'{ticker} Stock Price and Moving Averages')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()

# Main function to perform financial data analysis
def main():
    ticker = 'AAPL'  # Example ticker (Apple Inc.)
    period = '1y'    # Example period (1 year)

    # Step 1: Download stock data
    stock_data = download_stock_data(ticker, period)
    if stock_data.empty:
        print(f"Failed to download data for {ticker}.")
        return

    # Step 2: Calculate moving averages
    stock_data = calculate_moving_averages(stock_data)

    # Step 3: Plot the data
    plot_stock_data(stock_data, ticker)

# Run the main function
if __name__ == "__main__":
    main()